This notebook produces explanations using the counterfactual explainer

In [1]:
import sys
import json
import pickle
import warnings
import numpy as np
import pandas as pd
sys.path.append('../')
warnings.filterwarnings("ignore")
import fatf.transparency.predictions.counterfactuals as fatf_cf

24-Apr-28 17:55:00 fatf.utils.array.tools INFO     Using numpy's numpy.lib.recfunctions.structured_to_unstructured as fatf.utils.array.tools.structured_to_unstructured and fatf.utils.array.tools.structured_to_unstructured_row.


In [2]:
# load, clean and shape the data
train_data = pd.read_pickle('../data/explainability_input/train_data.pkl')
x_train_data = train_data.drop(columns=['id', 'date', 'cluster'])
x_train_data = x_train_data.drop(columns=x_train_data.columns[15])
y_train_data = train_data['cluster']
test_data = pd.read_pickle('../data/explainability_input/test_data.pkl')
x_test_data = test_data.drop(columns=['id', 'date', 'cluster'])
y_test_data = test_data['cluster']

In [3]:
# load the model
with open('../models/svc.pkl', 'rb') as file:  
    model = pickle.load(file)

In [4]:
# initialize the counterfactual explainer
cf_explainer = fatf_cf.CounterfactualExplainer(
    model=model,
    dataset=x_train_data.values,  # 'it is advised to use the same dataset as for the training of the model object'
    categorical_indices=[],
    default_numerical_step_size=0.1)

In [5]:
# extract parts of the counterfactual explainations
instance = 0  #TODO: add manually the index of the instance you want to explain
instance_x = x_test_data.iloc[instance, :].values
instance_y = y_test_data.iloc[instance] 
exp = cf_explainer.explain_instance(instance_x)
counterfactuals, distances, predictions = exp

In [6]:
# extract the raw local counterfactual explanation which is the most feasible to implement in a real-world scenario (aka smallest distance)
counterfactual = np.where(distances == np.min(distances))[0][0]
counterfactuals[counterfactual]

array([0.62292818, 0.        , 0.6       , 0.08113392, 0.01756253,
       0.0806846 , 0.        , 0.        , 0.        , 0.25470085,
       0.08304498, 0.92013889, 0.07407407, 0.00186482, 0.81046931,
       0.        , 0.02938176])

In [7]:
# visualize/textualize the local counterfactual explanation
counterfactuals_text = fatf_cf.textualise_counterfactuals(
    instance_x,
    counterfactuals=counterfactuals[counterfactual].reshape(1, -1),
    instance_class=instance_y)
print(counterfactuals_text)

Instance (of class *1*):
[0.62292818 0.         0.93680556 0.08113392 0.01756253 0.0806846
 0.         0.         0.         0.25470085 0.08304498 0.49513889
 0.07407407 0.00186482 0.81046931 0.         0.02938176]

Feature names: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

Counterfactual instance:
    feature *2*: *0.9368055555555556* -> *0.6000000000000001*
    feature *11*: *0.4951388888888889* -> *0.9201388888888888*


In [8]:
# store the counterfactual explanation in a json file
values = counterfactuals[counterfactual].tolist()
keys = x_train_data.columns.tolist()
cf_dict = dict(zip(keys, values))
with open('../data/explainability_output/local_0_counterfactual.json', 'w') as f:
    json.dump(cf_dict, f, indent=4)